**Load Packages**

In [42]:
# Importing necessary libraries

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import matplotlib.pyplot as plt # visualization
import matplotlib.pylab as plb # visualization
import warnings # ignore deprecation warnings etc.
warnings.filterwarnings("ignore")
import time # dealing with time data
from datetime import datetime, timedelta # dealing with time data
import datetime as dt # dealing with time data

**Load Data**

We have used matches and stats data

In [43]:
stats = pd.read_csv("582\\Project/stats/stats.csv")
matches = pd.read_csv("582\\Project/matches/matches.csv")

### Data Preprocessing

Matches DataFrame

**Filter English Premier League**

In [44]:
# Filter English Premier League Games Alone
matches = matches[matches["league_id"] == 148]

**Convert Epoch Column to Datetime Format**

dates were in epoch format and needs to be converted into datetime for interpretability

In [45]:
# Convert epoch column to datetime format
start = datetime(1970, 1, 1)  # Unix epoch start time
matches['datetime'] = matches.epoch.apply(lambda x: start + timedelta(seconds=x))
# Drop epoch
matches.drop("epoch",axis = 1, inplace = True)

# Split datetime column to date and time columns
matches['Date'] = matches['datetime'].dt.strftime('%d/%m/%Y')
matches['Time'] = matches['datetime'].dt.strftime('%H:%M:%S')

**Fill NA values with "not played"**

In [46]:
matches.match_hometeam_score = matches.match_hometeam_score.fillna("not_played_h")
matches.match_awayteam_score = matches.match_awayteam_score.fillna("not_played_a")

**Change Column Names of Matches DataFrame for convenience**

In [47]:
matches.rename(columns = {'match_hometeam_score':"home_score",'match_awayteam_score':"away_score" },inplace = True)

**Create a target column**

In [48]:
matches["match_result"] = np.where(matches['home_score'] == "not_played_h","Not Played",np.where(matches['home_score'] > matches['away_score'], "Home", np.where(matches['home_score'] == matches['away_score'],"Draw","Away")))

**Convert "not played" scores to 0 for calculation convenience**

In [49]:
matches.home_score = np.where(matches["home_score"] == "not_played_h","0",matches["home_score"])
matches.home_score = matches.home_score.astype(int)

matches.away_score = np.where(matches["away_score"] == "not_played_a","0",matches["away_score"])
matches.away_score = matches.away_score.astype(int)

### **Feature Engineering**

We have computed each feature for home and away. For some features we have computed them across seasons(*) and for some we have disregarded season information and used the data as a whole. Below are features we have included in our model and their explanations.

**1. Number of conceded goals for 2 seasons for home and away teams**

(*): We have partitioned data into seasons however, these seasons are partly artificial. We have considered 2019-2020 season as this season and 2 seasons before as last season<br>

Finding the number of goals conceded by home and away teams for each of the seasons defined. Conceded goals refer to number of goals a team has let in against the opposition.

In [50]:
# Last season end date
last_season_end = "2019-08-08 23:00:00"
# Split current matches dataframe into last season and this season
# Last Season
last_season_df = matches[matches["datetime"] < last_season_end]
# This season
this_season_df =  matches[matches["datetime"] > last_season_end]

In [51]:
#Last Season
a = pd.melt(last_season_df, id_vars = ['datetime',"match_id","match_hometeam_id","match_awayteam_id"], value_vars = ["home_score","away_score"], var_name = "Var1",value_name='Var1_value')
b = pd.melt(last_season_df, id_vars=['datetime',"match_id","home_score","away_score"], value_vars = ["match_hometeam_id","match_awayteam_id"],var_name='Var2', value_name='Var2_value')
last_season = a.merge(b,on = "match_id")
last_season = last_season[["datetime_x","match_id","Var1","Var1_value","Var2","Var2_value"]]

last_season.sort_values(["datetime_x","Var2_value"])

# Drop values that are duplicated but not needed
last_season =(last_season[~((last_season["Var2"] == "match_hometeam_id") & (last_season["Var1"] == "home_score"))])
last_season =(last_season[~((last_season["Var2"] == "match_awayteam_id") & (last_season["Var1"] == "away_score"))])


goals_last_season_dict_conceded = last_season.groupby("Var2_value")["Var1_value"].mean().to_dict()

#This Season
a = pd.melt(this_season_df, id_vars = ['datetime',"match_id","match_hometeam_id","match_awayteam_id"], value_vars = ["home_score","away_score"], var_name = "Var1",value_name='Var1_value')
b = pd.melt(this_season_df, id_vars=['datetime',"match_id","home_score","away_score"], value_vars = ["match_hometeam_id","match_awayteam_id"],var_name='Var2', value_name='Var2_value')
this_season = a.merge(b,on = "match_id")
this_season = this_season[["datetime_x","match_id","Var1","Var1_value","Var2","Var2_value"]]

this_season.sort_values(["datetime_x","Var2_value"])

# Drop values that are duplicated but not needed
this_season =(this_season[~((this_season["Var2"] == "match_hometeam_id") & (this_season["Var1"] == "home_score"))])
this_season =(this_season[~((this_season["Var2"] == "match_awayteam_id") & (this_season["Var1"] == "away_score"))])


goals_this_season_dict_conceded = this_season.groupby("Var2_value")["Var1_value"].mean().to_dict()

In [52]:
matches["last_season_goals_home_concede"] = matches["match_hometeam_id"].map(goals_last_season_dict_conceded)
matches["last_season_goals_away_concede"] = matches["match_awayteam_id"].map(goals_last_season_dict_conceded)
matches["this_season_goals_home_concede"] = matches["match_hometeam_id"].map(goals_this_season_dict_conceded)
matches["this_season_goals_away_concede"] = matches["match_awayteam_id"].map(goals_this_season_dict_conceded)

**2. Calculating attack or defense strength**

This feature is inspired by this blog post.(https://help.smarkets.com/hc/en-gb/articles/115001457989-How-to-calculate-Poisson-distribution-for-football-betting)
After home and away team's attack strength and defense strength we will use these features as a way of calculating home and away team's number of goals. To elaborate for a home team we need to calculate attack strength and then the defense strength of away team. Take the product of home team attack strength, away team defense strength and home teams average goals. Our main goal was the use last season's statistics however,as mentioned in introduction not all matches included are premier league matches for some teams. Instead we used whole data as one season

In [53]:
# Home
#Average goals scored at home = total home goals scored in the season / total number of home games
average_goals_scored_at_home = sum(matches["home_score"])/len(matches["home_score"])

# Away
#Average goals scored away = total away goals scored in the season / total number of away games
average_goals_scored_away = sum(matches["away_score"])/len(matches["away_score"])


The next step is to determine the average number of goals conceded per game - for both home and away teams - which is the opposite to the average goals scored per game.

In [54]:
# Average number of goals conceded by the home team
average_goals_conceded_by_home = average_goals_scored_away

# Average number of goals conceded by the away team
average_goals_conceded_away = average_goals_scored_at_home

**Calculating the home team's attack strength & away defense strength**

In [55]:
#There are two steps to calculating a home team's attack strength.
#Number of goals scored at home last season by the home team / number of home games played

# Home attack strength
ratio_home = matches.groupby("match_hometeam_id")["home_score"].rolling(5,min_periods = 0).mean().reset_index(drop = True)
hometeam_attack_strength_season_28 = ratio_home/average_goals_scored_at_home

# Away Defensive Strength
ratio_away = matches.groupby("match_awayteam_id")["away_score"].rolling(5,min_periods = 0).mean().reset_index(drop = True)
awayteam_defense_strength_season_28 = ratio_away/average_goals_conceded_away

Basically we have matches that do not belong to premier league but championship eg. Aston Villa joins premier league in 28th season but in our dataframe we see it in the previous 2 seasons. Also, teams should technically have the same number of matches in a season but I think some matches are removed because it was wanted there be an equality in team names through the dataset. We need to replace teams that do not belong to 27th season with their values so far for 28th season

In [56]:
matches["hometeam_attack_strength_season_28"] = hometeam_attack_strength_season_28
matches["awayteam_defense_strength_season_28"] = awayteam_defense_strength_season_28
hometeam_average_goals_dict = (matches.groupby("match_hometeam_id").sum()["home_score"]/ matches.groupby("match_hometeam_id").count()["home_score"]).to_dict()
matches["hometeam_average_goals_season_28"] = matches["match_hometeam_id"].map(hometeam_average_goals_dict)

**Projecting expected home team goals**

In [57]:
#Home team attack strength * away team defence strength * average number of home goals
matches["expected_hometeam_goals"] = matches["hometeam_attack_strength_season_28"] * matches["awayteam_defense_strength_season_28"] * matches["hometeam_average_goals_season_28"] 

**Calculating the away team's attack strength & home team's defensive strength**

We repeat the above steps for away team likewise

In [58]:
# Away attack strength
ratio_away = matches.groupby("match_awayteam_id")["away_score"].rolling(5,min_periods = 0).mean().reset_index(drop = True)
awayteam_attack_strength_season_28 = ratio_away/average_goals_scored_away

# Home Defensive Strength
ratio_home = matches.groupby("match_hometeam_id")["home_score"].rolling(5,min_periods = 0).mean().reset_index(drop = True)
hometeam_defense_strength_season_28 = ratio_home/average_goals_conceded_by_home

In [59]:
matches["awayteam_attack_strength_season_28"] = awayteam_attack_strength_season_28.values
matches["hometeam_defense_strength_season_28"] = hometeam_defense_strength_season_28.values
awayteam_average_goals_dict = (matches.groupby("match_awayteam_id").sum()["away_score"]/ matches.groupby("match_awayteam_id").count()["away_score"]).to_dict()
matches["awayteam_average_goals_season_28"] = matches["match_awayteam_id"].map(awayteam_average_goals_dict)

In [60]:
#Home team attack strength * away team defence strength * average number of home goals
matches["expected_awayteam_goals"] = matches["awayteam_attack_strength_season_28"] * matches["hometeam_defense_strength_season_28"] * matches["awayteam_average_goals_season_28"] 

Drop features that were used as part of the calculation of expected goals

In [61]:
matches.drop(['hometeam_attack_strength_season_28',
       'awayteam_defense_strength_season_28',
       'hometeam_average_goals_season_28', 
        'awayteam_attack_strength_season_28',
       'hometeam_defense_strength_season_28',
       'awayteam_average_goals_season_28'],axis = 1,inplace = True)

**Total Goals by Home and Away Teams**

Total goals scored by home and away teams 

In [62]:
#Total goals of all times
total_home_score = matches[["match_hometeam_id","home_score"]].groupby("match_hometeam_id").sum()
total_home_score_dict = total_home_score.to_dict()["home_score"]
total_away_score = matches[["match_awayteam_id","away_score"]].groupby("match_awayteam_id").sum()
total_away_score_dict = total_away_score.to_dict()["away_score"]
matches["total_home_score"] = matches["match_hometeam_id"].map(total_home_score_dict)
matches["total_away_score"] = matches["match_awayteam_id"].map(total_away_score_dict)

In [63]:
# Drop columns that are redundant
matches.drop(["match_status","match_live",
            "match_hometeam_halftime_score","match_awayteam_halftime_score",
            "match_hometeam_extra_score","match_awayteam_penalty_score",
            "match_hometeam_halftime_score","match_awayteam_halftime_score",
           "match_awayteam_extra_score","match_hometeam_penalty_score","league_id","Date","Time"],axis = 1,inplace= True)

**Take last 20 games's stats**

As an alternative to calculating win/lose streaks we tried to create features based on statistics of home and away games. We chose corner kicks, ball possession and red card. We based our selection criteria on the number of null values because some statistics contained too many null values and the chosen values due to their significance in a match. For example a large number of corner kicks indicates that a team plays well by enforcing corner kicks from the opponent team. 

Here we take the mean of the last 20 games for home and away teams in terms of their statistics mentioned above.

In [64]:
a = matches[["match_id","match_hometeam_id","match_awayteam_id"]].drop_duplicates()
a = a.set_index("match_id")
a = a.to_dict()
home_team_id_dict = a["match_hometeam_id"]
away_team_id_dict = a["match_awayteam_id"]

stats["match_hometeam_id"] = stats["match_id"].map(home_team_id_dict)
stats = stats[~stats["match_hometeam_id"].isnull()]
stats["match_hometeam_id"] = stats["match_hometeam_id"].astype(int)

stats["match_awayteam_id"] = stats["match_id"].map(away_team_id_dict).astype(int)


In [65]:
stats = stats[["match_id",
                            "home_BallPossession",
                            "away_BallPossession",
                            "home_CornerKicks",
                            "away_CornerKicks",
                            "home_RedCards",
                            "away_RedCards"]]

stats_matches = pd.merge(left = stats, right = matches,how = "right",right_on = "match_id",left_on = "match_id")
#stats_matches = stats_matches.fillna(0)
columns_to_change_home = ["home_CornerKicks", "home_RedCards"]
columns_to_change_away = ['away_CornerKicks','away_RedCards']
# Home corner kick
homekick_dict = stats_matches.groupby("match_hometeam_id").nth([x for x in range(-20,0)]).home_CornerKicks.mean(axis=0,level=0).to_dict()
stats_matches["home_kick_avg"] = stats_matches["match_hometeam_id"].map(homekick_dict)

# Away corner kick
awaykick_dict = stats_matches.groupby("match_awayteam_id").nth([x for x in range(-20,0)]).away_CornerKicks.mean(axis=0,level=0).to_dict()
stats_matches["away_kick_avg"] = stats_matches["match_awayteam_id"].map(awaykick_dict)

# Home Red Card
homered_dict = stats_matches.groupby("match_hometeam_id").nth([x for x in range(-20,0)]).home_RedCards.mean(axis=0,level=0).to_dict()
stats_matches["home_red_avg"] = stats_matches["match_hometeam_id"].map(homered_dict)

# Away Red Card

awayred_dict = stats_matches.groupby("match_awayteam_id").nth([x for x in range(-20,0)]).away_RedCards.mean(axis=0,level=0).to_dict()
stats_matches["away_red_avg"] = stats_matches["match_awayteam_id"].map(awayred_dict)

In [66]:
#Home Possession
stats_matches.home_BallPossession = stats_matches.home_BallPossession.astype("category")
stats_matches.away_BallPossession = stats_matches.away_BallPossession.astype("category")
stats_matches.home_BallPossession = stats_matches["home_BallPossession"].cat.add_categories(["00%"]).fillna("00%")
stats_matches.away_BallPossession = stats_matches.away_BallPossession.cat.add_categories(["000%"]).fillna("000%")
stats_matches.home_BallPossession = stats_matches.home_BallPossession.str[0:2]
stats_matches.away_BallPossession = stats_matches.away_BallPossession.str[0:2]
# Fill NA
#stats_matches.away_BallPossession = stats_matches.away_BallPossession.fillna(0)
#stats_matches.home_BallPossession = stats_matches.home_BallPossession.fillna(0)
# Convert to int
stats_matches.away_BallPossession  = stats_matches.away_BallPossession.astype(int)
stats_matches.home_BallPossession = stats_matches.home_BallPossession.astype(int)

homepos_dict = stats_matches.groupby("match_hometeam_id").nth([x for x in range(-20,0)]).home_BallPossession.mean(axis=0,level=0).to_dict()
stats_matches["home_pos"] = stats_matches["match_hometeam_id"].map(homepos_dict)

#Away Possesion

awaypos_dict = stats_matches.groupby("match_awayteam_id").nth([x for x in range(-20,0)]).away_BallPossession.mean(axis=0,level=0).to_dict()
stats_matches["away_pos"] = stats_matches["match_awayteam_id"].map(awaypos_dict)

# Points

We have partitioned data into 2 seasons and computed the sum of points throughout each seasons for home and away teams. Our point calculations were very close the points,with slight deviation, showcased by premier league however since our data also contained matches from EFL cup.

In [67]:
teamidllist=matches["match_hometeam_id"].unique().tolist()

In [68]:
last_season_end = "2019-08-08 23:00:00"

last_season_stats = stats_matches[stats_matches["datetime"] < last_season_end]
this_season_stats=  stats_matches[stats_matches["datetime"] > last_season_end]

new_dataframe = pd.DataFrame()

new_dataframe["teams"] = teamidllist
new_dataframe = new_dataframe.sort_values("teams",ascending = True).reset_index()

After we calculated points for home and away teams we have converted them into dictionaries and then summed these dictionaries based on their keys. Therefore, we have obtained a final dictionary that contains points for each team in season 2018-2019.

In [69]:
# LAST SEASON
last_season_stats["Home_Point"] = last_season_stats.match_result.apply(lambda x: 3 if x == "Home" else (1 if x== "Draw" else 0))
last_season_stats["Away_Point"] = last_season_stats.match_result.apply(lambda x: 3 if x == "Away" else (1 if x== "Draw" else 0))

home_dict = last_season_stats.groupby("match_hometeam_id").sum()[["Home_Point"]].to_dict()
home_dict_last = home_dict["Home_Point"]
away_dict = last_season_stats.groupby("match_awayteam_id").sum()[["Away_Point"]].to_dict()
away_dict_last = away_dict["Away_Point"]

from collections import Counter 

total_dict = Counter(home_dict_last) + Counter(away_dict_last) 

new_dataframe["old_season_points"] = new_dataframe.teams.map(total_dict)
new_dataframe = new_dataframe.sort_values("old_season_points", ascending = False)

After we calculated points for home and away teams we have converted them into dictionaries and then summed these dictionaries based on their keys. Therefore, we have obtained a final dictionary that contains points for each team in season 2019-2020.

In [70]:
# THIS SEASON
this_season_stats["Home_Point"] = this_season_stats.match_result.apply(lambda x: 3 if x == "Home" else (1 if x== "Draw" else 0))
this_season_stats["Away_Point"] = this_season_stats.match_result.apply(lambda x: 3 if x == "Away" else (1 if x== "Draw" else 0))

home_dict = this_season_stats.groupby("match_hometeam_id").sum()[["Home_Point"]].to_dict()
home_dict_this = home_dict["Home_Point"]
away_dict = this_season_stats.groupby("match_awayteam_id").sum()[["Away_Point"]].to_dict()
away_dict_this = away_dict["Away_Point"]

from collections import Counter 

total_dict = Counter(home_dict_this) + Counter(away_dict_this) 

new_dataframe["This_season_points"] = new_dataframe.teams.map(total_dict)
new_dataframe = new_dataframe.sort_values("This_season_points", ascending = False)

In [71]:
# drop redundant columns
stats_matches.drop(['home_BallPossession', 'away_BallPossession',
       'home_CornerKicks', 'away_CornerKicks', 'home_RedCards',
       'away_RedCards'], axis =1,inplace = True)

In [72]:
stats_matches["old_season_points_home"] = stats_matches.match_hometeam_id.map(home_dict_last)
stats_matches["old_season_points_away"] = stats_matches.match_awayteam_id.map(away_dict_last)

stats_matches["this_season_points_home"] = stats_matches.match_hometeam_id.map(home_dict_this)
stats_matches["this_season_points_away"] = stats_matches.match_awayteam_id.map(away_dict_this)

In [73]:
# we do not have a large population of null values in our selected statistics features hence we filled them with 0
stats_matches = stats_matches.fillna(0)

**About Insignificant Features we have created**

Above features are the ones that are crucial to our model. We have tried and manually discarded features based on their RPS score. These features include win/lose streak, goal difference, cumulative sum of the goals throughout a team's past games.When included they have not provided a decrease in RPS score.

**Model**

For getting probabilities for the upcoming round of matches. We have set a date interval that we manually intervened before every round. We have tried fitting our model with Multi-class Logistic Regression, Linear Discriminant Analysis,Decision Tree, Random Forest and Stochastic Gradient Boosting. Out of these algorithms Multi-class Logistic Regression gave the lowest RPS scores. However, Linear Discriminant Analysis for some rounds gave lower bias but variance of Multi-class Logistic Regression was lower providing us with a more robust model.

In [74]:
# NOW
from datetime import datetime

now = datetime.now()
#now_formatted = now.strftime("%Y-%m-%d %H:%M:%S")

# choose the date interval you'd like to test on
start_date = "2019-11-30 23:00:00"
end_date = "2020-01-04 23:00:00"

# Split into train and test data

train = stats_matches[stats_matches["datetime"]<start_date]
test = stats_matches[(stats_matches["datetime"]>start_date) & (stats_matches["datetime"]<=end_date)]

In [75]:
# Drop insignificant columns from training and test datasets

train_x = train.drop(["match_result","datetime","match_id",'match_hometeam_name', 'match_awayteam_name'],axis = 1)
target = train["match_result"]
test_for_names = test.copy()
test_x = test.drop(["match_result","datetime","match_id",'match_hometeam_name', 'match_awayteam_name'],axis = 1)

In [76]:
from sklearn.linear_model import LogisticRegression
# Logistic Regression
model = LogisticRegression(multi_class='multinomial',solver ='newton-cg').fit(train_x,target)

model.fit(train_x,target)
#predictions = model.predict(test_x)

# Obtain probabilites
probabilities = model.predict_proba(test_x)

probabilities_df = pd.DataFrame(probabilities,columns=model.classes_)

In [77]:
test_results = pd.concat([test_for_names.reset_index(),probabilities_df.reset_index()],axis = 1)
test_results = test_results[['Home','Draw','Away','match_awayteam_id', "match_hometeam_id","match_id",'match_hometeam_name', 'match_awayteam_name']]
test_results.to_csv("round_x.csv")

In [78]:
round_14 = pd.read_csv("582\\Project/rounds_RPS/round_14.csv")
round_15 = pd.read_csv("582\\Project/rounds_RPS/round_15.csv")
round_16 = pd.read_csv("582\\Project/rounds_RPS/round_16.csv")
round_17 = pd.read_csv("582\\Project/rounds_RPS/round_17.csv")
round_18 = pd.read_csv("582\\Project/rounds_RPS/round_18.csv")
round_19 = pd.read_csv("582\\Project/rounds_RPS/round_19.csv")
round_20 = pd.read_csv("582\\Project/rounds_RPS/round_20.csv")
round_21 = pd.read_csv("582\\Project/rounds_RPS/round_21.csv")


total_rounds = pd.concat([round_14,round_15,round_16,round_17,round_18,round_19,round_20,round_21],axis =0,ignore_index = True)

In [79]:
total_rounds = total_rounds.merge(matches, how = "inner", on ="match_id")[["match_id","Home","Draw","Away","match_result"]]

In [80]:
def rps (p1,p0,p2,result):
    if result =="Home":
        y_1=1;y_x=0;y_2=0
    elif result =="Draw":
        y_1=0;y_x=1;y_2=0
    elif result =="Away":
        y_1=0;y_x=0;y_2=1
    rps = 1/2*(  (p1 - y_1)**2 + (p1 + p0  - y_1 - y_x )**2 + (p1 + p0 + p2  - y_1 - y_x - y_2)**2)
    return rps

In [81]:
total_rounds["RPS"] = np.vectorize(rps)(total_rounds["Home"],total_rounds["Draw"],total_rounds["Away"],total_rounds["match_result"])

Our final results for all 8 rounds can be seen in the table below:

In [82]:
total_rounds["RPS"].describe()

count    75.000000
mean      0.222586
std       0.159870
min       0.000286
25%       0.128955
50%       0.184766
75%       0.312888
max       0.677117
Name: RPS, dtype: float64

As can be seen our RPS values are highly variable due to unpredictable nature of the games.